In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import time
import random

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 랜덤 대기 시간 설정
random_sec = random.uniform(1, 3)

# 웹드라이버 초기화 및 웹사이트 열기
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
url = "https://unipass.customs.go.kr/clip/index.do"
driver.get(url)

# 페이지가 완전히 로드될 때까지 기다림
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="TOPMENU_LNK_M_ULS0200000000"]/span')))

# 텍스트를 저장할 리스트 초기화
texts = []

hover_element = driver.find_element(By.XPATH, '//*[@id="TOPMENU_LNK_M_ULS0200000000"]/span')
click_element = driver.find_element(By.XPATH, '//*[@id="TOPMENU_LNK_UI-ULS-0201-002Q"]/span')

actions = ActionChains(driver)
actions.move_to_element(hover_element).perform()  # 마우스를 hover_element 위로 이동
actions.click(click_element).perform()  # click_element 클릭

time.sleep(5)  # 페이지가 완전히 로드될 때까지 기다림

# 원하는 요소를 클릭
click_element = driver.find_element(By.XPATH, '/html/body/div[3]/section[2]/div[2]/div/div/form/div/table/tbody/tr[1]/td[2]/a')
click_element.click()

time.sleep(random_sec)  # 페이지가 완전히 로드될 때까지 기다림

for i in range(78, 98):
    xpath = f"/html/body/div[3]/section[2]/div[2]/div/form/div[2]/div/div/div/p[{i}]/a"
    p_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, xpath)))
    
    try:
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, xpath))).click()
    except StaleElementReferenceException:
        # 요소가 Stale 상태라면 다시 찾아서 클릭
        driver.execute_script("arguments[0].click();", p_element)
        

    # 페이지 로딩 대기 및 필요한 작업 수행
    time.sleep(random_sec)
    
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#tblLstBody tr'))) # 요소가 페이지의 DOM에 존재하는지 확인
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    rows = soup.select('#tblLstBody tr')    
#     # 데이터 수집
    for row in rows:
        number = row.select_one('td:nth-of-type(1) a').text
        korean_name = row.select_one('td:nth-of-type(2) a').text
        english_name = row.select_one('td:nth-of-type(3) a').text
        texts.append((number, korean_name, english_name))
print(texts)

[('7801', '납의 괴(塊)', 'Unwrought lead.'), ('7802', '납의 웨이스트(waste)와 스크랩(scrap)', 'Lead waste and scrap.'), ('7804', '시트(sheet)ㆍ스트립ㆍ박(箔)[두께(보강재의 두께는 제외한다)가 0.2밀리미터 이하인 것으로 한정한다]', 'Sheets, strip and foil of a thickness (excluding any backing) not exceeding 0.2 ㎜'), ('7806', '납으로 만든 그 밖의 제품', 'Other articles of lead.'), ('7901', '아연의 괴(塊)', 'Unwrought zinc.'), ('7902', '아연의 웨이스트(waste)와 스크랩(scrap)', 'Zinc waste and scrap.'), ('7903', '아연의 더스트(dust)ㆍ가루ㆍ플레이크(flake)', 'Zinc dust, powders and flakes.'), ('7904', '아연의 봉ㆍ프로파일(profile)ㆍ선(線)', 'Zinc bars, rods, profiles and wire.'), ('7905', '아연의 판ㆍ시트(sheet)ㆍ스트립ㆍ박(箔)', 'Zinc plates, sheets, strip and foil.'), ('7907', '아연으로 만든 그 밖의 제품', 'Other articles of zinc.'), ('8001', '주석의 괴(塊)', 'Unwrought tin.'), ('8002', '주석의\xa0웨이스트(waste)와 스크랩(scrap)', 'Tin waste and scrap.'), ('8003', '주석의 봉ㆍ프로파일(profile)ㆍ선(線)', 'Tin bars, rods, profiles and wire.'), ('8007', '주석으로 만든 그 밖의 제품', 'Other articles of tin.'), ('8101', '텅스텐(볼프람)과 그 제품[웨이스트(waste)와 스크랩(scrap)

In [ ]:
# 첫 번째 df 생성
import pandas as pd
df = pd.DataFrame(texts, columns=['번호', '한글명', '영문명'],dtype=str)

In [7]:
# 두 번째 df 생성
df2 = pd.DataFrame(texts, columns=['번호', '한글명', '영문명'],dtype=str)

In [10]:
# df, df2 결합
final_df = pd.concat([df,df2])

In [12]:
final_df.to_csv('HSCODE_호.csv',index=False)